****
# Decorators, Closures and Wrappers in Python
****

### About this notebook
Documention prepared by **Jesus Perez Colino**.
Version 0.1, Released 25/11/2014, Alpha

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.
- **Summary**: This notebook is summary of **decorators** in Python. At the end of the notebook, two examples of memoization with decorators are showed.
- **Reproducibility conditions**: the content in this technical note can be reproduce in your computer under the following versions of the Python's packages: 

In [4]:
import IPython
import numpy as np
from sys import version 
print ' Reproducibility conditions for this notebook '.center(85,'-')
print ('Python version:     ' + version).center(85)
print ('Numpy version:      ' + np.__version__)
print 'IPython version:    ' + IPython.__version__
print '-'*85

-------------------- Reproducibility conditions for this notebook -------------------
Python version:     2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Sep 15 2015, 14:29:08) 
[GCC 4.2.1 (Apple Inc. build 5577)]
Numpy version:      1.9.2
IPython version:    4.0.0
-------------------------------------------------------------------------------------


## Introduction to Decorators

When dealing with a large codebase, it’s very common to have a set of tasks that need to be performed by many different functions, usually before or after doing something more specific to the function at hand. The nature of these tasks is as varied as the projects that use them, but here are some of the more common examples of where decorators are used:

- Access control 
- Cleanup of temporary objects
- Error handling
- Caching
- Logging

In all of these cases, there’s some boilerplate code that needs to be executed before or after what the function’s really trying to do. Rather than copying that code into each function, it’d be better if it could be written once and simply applied to each function that needs it. This is where decorators come in.

Technically, decorators are just simple functions designed with one purpose: *accept a function and return a function*. The function returned can be the same as the one passed in or it could be completely replaced by something else along the way. 

The most common way to apply a decorator is using a special syntax designed just for this purpose. Here’s how you could apply a decorator designed to suppress any errors during the execution of a function:


In [ ]:
import datetime
from my_package import suppress_errors

@suppress_errors
def log_error(message, log_file='errors.log'):
    """Log an error message to a file."""
    log = open(log_file,'w')
    log.write('%s\t%s\n' % (datetime.datetime.now(), message))


This syntax tells Python to pass the `log_error()` function as an argument to the `suppress_errors()` function, which then returns a replacement to use instead. It’s easier to understand what happens behind the scenes by examining the process used in older versions of Python, before the `@` syntax was introduced in Python 2.4:

In [ ]:
import datetime
from my_package import suppress_errors

def log_error(message, log_file='errors.log'):
    """Log an error message to a file."""
    log = open(log_file, 'w')
    log.write('%s\t%s\n' % (datetime.datetime.now(), message))
    
log_error = suppress_errors(log_error)

To understand what commonly goes on inside decorators like log_error(), it’s necessary to first examine one of the most misunderstood and underutilized features of Python, and many other languages as well: **closures**

## Closures
In a nutshell, *a closure is a function that’s defined inside another function but is then passed outside that function where it can be used by other code*. There are some other details to learn as well, but it’s still fairly abstract at this point, so here’s a simple example of a closure:

In [11]:
def multiply_by(factor):
    '''Return a function that multiplies values by the given factor'''
    def multiply(value):
        '''Multiply the given value by the factor already provided'''
        return value * factor
    return multiply

times2 = multiply_by(2)
print(times2(2))

4


When you call `multiply_by()` with a value to use as a multiplication factor, the inner `multiply()` gets returned to be used later on. 

Here’s how it would actually be used, which may help explain why this is useful. If you key in the previous code line by line from a Python prompt, the following would give you an idea as well about how this works:

In [14]:
times2 = multiply_by(2)
print times2(5)
print times2(10)
print times2(100)

10
20
200


In [15]:
times10 = multiply_by(10)
print times10(5)
print times10(10)
print times10(100)

50
100
1000


## Wrappers
Closures play an important role in the construction of wrappers, the most common use of decorators. **Wrappers** are *functions designed to contain another function, adding some extra behavior before or after the wrapped function executes*. 

In the context of the closure discussion, a **wrapper** *is the inner function,* while the wrapped function is passed in as an argument to the outer function.

In [ ]:
def suppress_errors(func):
    """Automatically silence any errors that occur within a function"""

    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception:
            pass
        
    return wrapper

Notice that here, the **decorator** `suppress_errors()` takes a function as its only argument, which isn’t executed until the inner wrapper function executes. By returning the wrapper instead of the original function, we form a **closure**, which allows the same function handle to be used even after `suppress_errors()` is done.

Because the `wrapper` has to be called as if it were the original function, regardless of how that function was defined, *it must accept all possible argument combinations*. This is achieved by using *variable positional and keyword arguments* together and passing them straight into the original function internally. This is a very common practice with wrappers because it allows maximum flexibility, without caring what type of function it’s applied to.

The actual work in the wrapper is quite simple: just execute the original function inside a `try/except block` to catch any errors that are raised. In the event of any errors, it just continues merrily along, implicitly returning None instead of doing anything interesting. It also makes sure to return any value returned by the original function, so that everything meaningful about the wrapped function is maintained.

In this case, the wrapper function is fairly simple, but the basic idea works for many more complex situations as well. There could be several lines of code both before and after the original function is called, perhaps with some decisions about whether it is called at all. Authorization wrappers, for instance, will typically return or raise an exception without ever calling the wrapped function, if the authorization failed for any reason.

In [17]:
import functools
def suppress_errors(func):
    """Automatically silence any errors that occur within a function"""
    @functools.wrapper(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception:
            pass

    return wrapper

This layering allows `suppress_errors()` to accept arguments prior to being used as a decorator, but it removes the ability to call it without any arguments. Because that was the previous behavior, we’ve now introduced a backward incompatibility. The closest we can get to the original syntax is to actually call `suppress_errors()` first, but without any arguments.

Here’s an example function that processes updates files in a given directory. This is a task that’s often performed on an automated schedule, so that if something goes wrong, it can just stop running and try again at the next appointed time:

In [ ]:
import datetime
import os
import time
from my_package import suppress_errors

@suppress_errors()
def process_updated_files(directory, process, since=None):
    """
    Processes any new files in a `directory` using the `process` function.
    If provided, `since` is a date after which files are considered updated.

    The process function passed in must accept a single argument: the absolute
    path to the file that needs to be processed.
    """
    
    if since is not None:
        # Get a threshold that we can compare to the modification time later
        threshold = time.mktime(since.timetuple()) + since.microsecond / 1000000
    else:
        threshold = 0
        
    for filename in os.listdir(directory):
        path = os.path.abspath(os.path.join(directory, filename))
        if os.stat(path).st_mtime > threshold:
            process(path)

## Example of Decorator: Memoization

To demonstrate how decorators can copy out common behavior into any function, let us consider what could be done to improve the efficiency of deterministic functions. Given such a function, it should be possible to cache the results of a given function call, so if it’s called with the same arguments again, the result can be looked up without having to call the function again.

Using a cache, a decorator can store the result of a function using the argument list as its key. Dictionaries or lists can’t be used as keys in a dictionary, so only positional arguments can be taken into account when populating the cache. Thankfully, most functions that would take advantage of memoization are simple mathematical operations, which are typically called with positional arguments anyway:


In [72]:
def memoize(func):
    """
    Cache the results of the function so it doesn't need to be called
    again, if the same arguments are provided a second time.
    """
    cache ={}
    @functools.wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        
        print('First time calling %s()' % func.__name__)
        
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

Now, whenever you define a deterministic function, you can use the `memoize()` decorator to automatically cache its result for future use. Here’s how it would work for some simple mathematical operations. Again, given you keyed in the aforelisted stub, try the following:

In [73]:
@memoize
def multiply(x, y):
    return x * y

In [74]:
multiply(6, 7)

First time calling multiply()


42

In [75]:
multiply(6, 7)

42

In [76]:
@memoize
def factorial(x):
    result = 1
    for i in range(x):
        result *= i+1
    return result

In [77]:
factorial(10)

First time calling factorial()


3628800

In [78]:
factorial(10)

3628800

Based in the same idea, but building the `@memoized` decorator using a class:

In [79]:
import collections
import functools

class memoized(object):
    def __init__(self, func):
        self.func = func
        self.cache = {}    
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable: a dict, or a list, for instance.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            print('First time calling %s()' % self.func.__name__)
            self.cache[args] = value
            return value
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [80]:
@memoized
def factorial(x):
    result = 1
    for i in range(x):
        result *= i+1
    return result

In [81]:
factorial(10)

First time calling factorial()


3628800

In [82]:
factorial(10)

3628800